In [1]:
print("OM NAMOH BHAGWATE VASUDEVAYE NAMAH")


OM NAMOH BHAGWATE VASUDEVAYE NAMAH


In [2]:
import os
os.chdir("./")
%pwd


'/Users/yash_new/Desktop/AI-Medical-Chatbot/research'

In [3]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


/opt/anaconda3/envs/medibot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    return loader.load()

extracted_data = load_pdf_files("data")
len(extracted_data)


incorrect startxref pointer(1)
parsing for Object Streams


759

In [16]:
from typing import List
from langchain_core.documents import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs = []

    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )

    return minimal_docs

minimal_docs = filter_to_minimal_docs(extracted_data)
len(minimal_docs)


759

In [17]:
def text_split(minimal_docs):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    return splitter.split_documents(minimal_docs)

texts_chunk = text_split(minimal_docs)
len(texts_chunk)


4110

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    return HuggingFaceEmbeddings(
        model_name="BAAI/bge-small-en-v1.5"
    )

embedding = download_embeddings()
len(embedding.embed_query("Hello World"))


/var/folders/zn/2rght4q50893h63h7nxfbkjm0000gn/T/ipykernel_11144/2033944873.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(


384

In [19]:
from dotenv import load_dotenv
load_dotenv()

import os
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "medical-chatbot"
existing_indexes = [i["name"] for i in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)
print("Pinecone index ready:", index_name)


Pinecone index ready: medical-chatbot


In [23]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)


In [24]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

retriever.invoke("What is Acne?")


[Document(id='c6244433-59aa-4314-89ed-1a03043aa7b1', metadata={'source': 'data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND copy.pdf'}, page_content='Patch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-\ntinued until the patient experiences an allergic\nreaction where the irritant was applied to the skin.\nRash—A spotted, pink or red skin eruption that\nmay be accompanied by itching and is caused by\ndisease, contact with an allergen, food ingestion,\nor drug reaction.\nUlcer—An open sore on the skin, resulting from\ntissue destruction, that is usually accompanied by\nredness, pain, or infection.\nPatients who have a history of dermatitis should\nremove their rings before washing their hands. They\nshould use bath oils or glycerine-based soaps and bathe\nin lukewarm saltwater.\

In [25]:
from langchain_groq import ChatGroq

chatModel = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    api_key=os.getenv("GROQ_API_KEY")
)

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
response = rag_chain.invoke(
    {"input": "What is the Treatment of Acne?"}
)

print(response["answer"])
